In [1]:
import setGPU

import os, sys
import scipy.io as sio
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import json
import multiprocessing.pool

sys.path.append('..')

from extractEmbeddings import featureExtractor
from utils import datasetUtils
from extractEmbeddings import fvsOperations

setGPU: Setting GPU to: 1


/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3

In [2]:
testDataPath = '../../../data/movie_titles/testUncompressed/' #test cases data path
titlesPath = '../../../data/movie_titles/all_titles_uncompressed/' # all movie titles path

In [3]:
dataUtils = datasetUtils()
fvsOps = fvsOperations()

In [4]:
"""
Use pre-trained models
"""
featExt = featureExtractor(1000, "ENB7")
model = featExt.pretrainedModel()

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
def testAggFvsStore():
    for title in os.listdir(titlesPath):
        titlePath = os.path.join(titlesPath, title)
        #print(titlePath)
        matFile = title+'ENB5.mat'
        matFilePath = os.path.join(titlePath, matFile)
        dMatFile = title+'AggregateENB5'+'.mat'
        dstMatFile = os.path.join(titlePath, dMatFile)
        fvsOps.aggregateFvsStore(matFilePath, dstMatFile)

In [ ]:
testAggFvsStore()

In [6]:
"""
test the load aggregate function
"""
def testLoadAggMatFile():
    for title in os.listdir(titlesPath):
        titlePath = os.path.join(titlesPath, title)
        matFilePath = titlePath+'/'+title+'AggregateENB7.mat' #166AggregateENB7.mat
        if not os.path.exists(matFilePath):
            print(matFilePath)
        featureVs = dataUtils.loadAggMatFile(matFilePath)

In [7]:
def displayTestAndMatchTitleFrames(testCaseFileNames, fileNames, titleCurrInd):

    _, axs = plt.subplots(len(testCaseFileNames), 2, figsize=(20, 20))
    axs = axs.flatten()
    testCount = 0
    titleCount = 0
    filenames = fileNames[titleCurrInd:(titleCurrInd+len(testCaseFileNames))]
    for ind, ax in enumerate(axs):
        if ind % 2 == 0:
            img = cv.imread(testCaseFileNames[testCount])
            ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
            ax.axis('off')
            testCount += 1
        else:
            filePath = '../'+filenames[titleCount].rstrip()
            img = cv.imread(filePath)
            ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
            ax.axis('off')
            titleCount += 1
            
    plt.show()

In [8]:
def testFramesFvs(path, model, featExt, isSignedMagSep, isOriginal):
    testFvs, testFileNames = featExt.extractFeaturesTitle(testCasePath, model) #extract features from test case folder
    if isOriginal:
        return testFvs
    arrTestFvs = np.asarray(testFvs).squeeze() #convert list to nd.array    
    if isSignedMagSep == True:
        arrTestFvsCombined = np.apply_along_axis(fvsOps.signedmag, 0, arrTestFvs) #combined feature vectors
    else:
        arrTestFvsCombined = np.linalg.norm(arrTestFvs, axis=0)
        
    return arrTestFvsCombined

In [9]:
def getArrFvsDB(path, title, isSignedMag, isOriginal):
    if isSignedMag:
        matFile = title+'AggregateENB7.mat'
        matFilePath = os.path.join(path, matFile)
        if not os.path.exists(matFilePath):
            print("mat file does not exists",matFilePath)
        arrFvsDB = dataUtils.loadAggMatFile(matFilePath)
    else:
        matFileName = title + '.mat'
        matFilePath = os.path.join(path, matFileName)
        if not os.path.exists(matFilePath):
            print("mat file does not exists",matFilePath)
        fvsDB, fileNamesDB = dataUtils.loadMatFile(matFilePath)
        if isOriginal:
            return fvsDB
        arrFvsDB = np.asarray(fvsDB).squeeze()
            
    return arrFvsDB

In [10]:
def findBestMatch(testCasePath, titlesPath, model, featExt):
    isSignedMagSep = False
    isOriginal = False

    matchedTitle = ''
    matchedInd = 0    

    minScore = 99999 # set initial min score as high
    maxVal = 0
    
    testFvs = testFramesFvs(testCasePath, model, featExt, isSignedMagSep, isOriginal)
    
    for title in os.listdir(titlesPath): #loading feature maps from each of the movie titles
        titlePath = os.path.join(titlesPath, title)
        
        arrFvsDB = getArrFvsDB(titlePath, title, isSignedMagSep, isOriginal)
        
        if isSignedMagSep == True:
            currScore, currIndDB = fvsOps.compareCombinedFvsFromFile(testFvs, arrFvsDB)
        else:
            currScore, currIndDB = fvsOps.compareCombinedFvs(arrFvsDB, testFvs)
            #currScore, currIndDB = fvsOps.compareFvs(fvsDB, testfvs)
            
        if currScore < minScore:
            minScore = currScore            
            matchedTitle = title
            matchedInd = currIndDB
            #print("min score, file index and the title name", minScore, currIndDB, title)
                
    #displayTestAndMatchTitleFrames(testFileNames, fileNamesDB, currIndDB)"""
    
    return matchedTitle, matchedInd

In [ ]:
"""
Use custom models
"""
#modelPath = '../../model/250919/weights-improvement-298-0.98.h5' # model path
#layerName = 'dense2' #layer name
#model = getModel(modelPath, layerName) #load model

#testCaseCount = 0
for testCase in os.listdir(testDataPath): #load each test case
    testCasePath = testDataPath + testCase
    
    predictedTitle, index = findBestMatch(testCasePath, titlesPath, model, featExt)
    aTitle, aTime = dataUtils.parseTestCaseJsonFile(testCasePath)
    
    at = int(aTime)
    timeDiff = index - at

    if predictedTitle == aTitle:
        print("Predicted title is correct, title: {}, index: {}, actual time: {}, time diff: {}".format(predictedTitle, index*24, aTime, timeDiff))
    else:
        print("Predicted title is wrong, predictedTitle: {}, actualtitle: {}".format(predictedTitle, aTitle))
    
    #testCaseCount += 1
    #if testCaseCount > 0:
        #break

Predicted title is correct, title: 5454, index: 479808, actual time: 832, time diff: 19160
Predicted title is correct, title: 28362, index: 687744, actual time: 1194, time diff: 27462
Predicted title is correct, title: 180, index: 214272, actual time: 372, time diff: 8556
Predicted title is correct, title: 166, index: 976896, actual time: 1702, time diff: 39002
Predicted title is correct, title: 231, index: 2222208, actual time: 3858, time diff: 88734
Predicted title is wrong, predictedTitle: 789880, actualtitle: 81671
Predicted title is wrong, predictedTitle: 790072, actualtitle: 81672
